In [1]:
import pandas as pd
import numpy as np

In [58]:
# df = pd.read_csv("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs/2021/stop_times.txt")
df = pd.read_csv("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs/2021/calendar.txt")


In [146]:
r = df.loc[1]

In [121]:
df1 = pd.DataFrame(active_dates,columns=["date"])
df1["service_id"] = r["service_id"]
df1["exception_type"] = 1
df1

,date,service_id,exception_type
0,20201228,16728,1
1,20201224,16728,1
2,20201231,16728,1
3,20201227,16728,1
4,20210103,16728,1


In [144]:
def serviceToDatesHelper(r): 
    '''Helper function that returns a list of date strings, given a calendar row
        with weekdays, start and end dates
    '''
    # day suffixes to be able to use pd.date_range to get a list of dates
    day_suff = ["MON","TUE","WED","THU","FRI","SAT","SUN"]
    # get relevant information from the row
    weekdays = [i for i,v in enumerate(r[0:7]) if v!=0]
    start = dt.datetime.strptime(str(r.start_date),"%Y%m%d").date()
    end = dt.datetime.strptime(str(r.end_date),"%Y%m%d").date()
    
    # generate the date strings
    active_dates = [
        i for l in [
            # get a date range of all dates...
            pd.date_range(start=start,end=end,freq="W-"+day_suff[d]
                          # ...for each weekday contained in service
                      ).strftime("%Y%m%d") for d in weekdays
            ] for i in l] #flatten this list situation
    
    return(active_dates)

def serviceToExcHelper(r):
    '''Helper function that returns a calendar_dates-like dataframe when
        applied to rows of calender.txt
    '''
    active_dates = serviceToDatesHelper(r)
    df = pd.DataFrame(active_dates,columns=["date"])
    df["service_id"] = r["service_id"]
    df["exception_type"] = 1
    
    return(df)

def serviceToExcHelperLoopy(r):
    '''Same as above, but not returning a dataframe, rather two same-length lists
        (exc-type can be added later, since it is always 1)
    '''
    active_dates = serviceToDatesHelper(r)
    service_id = [r["service_id"]]*len(active_dates)
    
    return(service_id, active_dates)

In [143]:
df2 = pd.DataFrame()
for _, row in df.iterrows():
    df2 = df2.append(serviceToExcHelper(row))
    
df2

,date,service_id,exception_type
0,20201228,16728,1
1,20201224,16728,1
2,20201231,16728,1
3,20201227,16728,1
4,20210103,16728,1
...,...,...,...
3,20211209,34130,1
4,20211210,34130,1
5,20211204,34130,1
6,20211211,34130,1


In [180]:
def calendarToDates(df):
    '''Takes a calendar.txt-df and returns a calendar_dates.txt-style df'''
    ids, dates = [], []
    for _, row in df.iterrows():
        i, d = serviceToExcHelperLoopy(row)
        ids += i
        dates += d

    df_out = pd.DataFrame({'service_id': ids,
                          'exception_type': [1] * len(ids),
                          'date': dates})
    df_out.date = df_out.date.astype(int)
    return(df_out)

In [157]:
exc_df = pd.read_csv("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs/2021/calendar_dates.txt")

In [158]:
exc_df

,service_id,exception_type,date
0,1,1,20210620
1,1,1,20210627
2,1,1,20210704
3,1,1,20210711
4,1,1,20210718
...,...,...,...
174483,9996,1,20210807
174484,9996,1,20210810
174485,9996,1,20210811
174486,9996,1,20210812


In [160]:
sum(exc_df.duplicated(exc_df.columns))

0

In [163]:
sum(exc_df.duplicated(["service_id",'date']))

0

Gut, wenigstens ist hier schonmal nichts dupliziert...

In [172]:
comp_df = df3.merge(exc_df, on=["service_id",'date'], how='outer')

In [173]:
comp_df[(comp_df.exception_type_x==1) & (comp_df.exception_type_y==1)]

,service_id,date,exception_type_x,exception_type_y
1,16728,20201224,1.0,1.0


eine einzige doppelte addition...

In [174]:
comp_df[(comp_df.exception_type_x!=1) & (comp_df.exception_type_y==2)]

,service_id,date,exception_type_x,exception_type_y


In [177]:
comp_df[(comp_df.exception_type_y==2)]

,service_id,date,exception_type_x,exception_type_y
52,10379,20210411,1.0,2.0
60,14568,20210411,1.0,2.0
67,170,20210404,1.0,2.0
93,24064,20210418,1.0,2.0
94,24064,20210425,1.0,2.0
...,...,...,...,...
1224412,32281,20211120,1.0,2.0
1224417,32281,20211121,1.0,2.0
1224554,14685,20211113,1.0,2.0
1224555,14685,20211120,1.0,2.0


keine subtraktion auf nicht vorhandenen Tagen.

2021 ist okay! Wie sieht es in 20 aus?

In [178]:
df20 = pd.read_csv("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs/2020/calendar.txt")

In [ ]:
df20_dates = calendarToDates(df20)

In [184]:
exc20_df = pd.read_csv("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs/2020/calendar_dates.txt")

In [185]:
sum(exc20_df.duplicated(exc20_df.columns))

0

In [186]:
sum(exc20_df.duplicated(["service_id",'date']))

0

Gut, wenigstens ist hier schonmal nichts dupliziert...

In [188]:
comp_df = df20_dates.merge(exc20_df, on=["service_id",'date'], how='outer')

In [189]:
comp_df[(comp_df.exception_type_x==1) & (comp_df.exception_type_y==1)]

,service_id,exception_type_x,date,exception_type_y


gar keine doppelte addition...

In [190]:
comp_df[(comp_df.exception_type_x!=1) & (comp_df.exception_type_y==2)]

,service_id,exception_type_x,date,exception_type_y


In [191]:
comp_df[(comp_df.exception_type_y==2)]

,service_id,exception_type_x,date,exception_type_y
60,12690,1.0,20200112,2.0
63,12690,1.0,20200202,2.0
72,12690,1.0,20200405,2.0
161,22558,1.0,20200105,2.0
162,22558,1.0,20200112,2.0
...,...,...,...,...
1621076,12247,1.0,20201125,2.0
1621080,12247,1.0,20201126,2.0
1621084,12247,1.0,20201127,2.0
1621431,10511,1.0,20201128,2.0


Puh. Sieht alles okay aus.

In [202]:
remain = comp_df[((comp_df.exception_type_x==1) & (comp_df.exception_type_y!=2))]
added = comp_df[((comp_df.exception_type_x!=1) & (comp_df.exception_type_y==1))]
actual_df = comp_df[((comp_df.exception_type_x==1) & (comp_df.exception_type_y!=2)) | # regular trips
                    ((comp_df.exception_type_x!=1) & (comp_df.exception_type_y==1))   # exceptional additions
                   ]

In [208]:
(len(df20_dates)-len(remain))/len(df20_dates)

0.05743955945696781

In [210]:
(len(added))/len(df20_dates)

0.07432498681885874

In [211]:
len(actual_df)/len(df20_dates)

1.016885427361891

In [215]:
actual_df

,service_id,exception_type_x,date,exception_type_y
0,13867,1.0,20191222,NaN
1,13867,1.0,20191229,NaN
2,13867,1.0,20200105,NaN
3,13867,1.0,20200112,NaN
4,13867,1.0,20200119,NaN
...,...,...,...,...
1742158,9999,NaN,20201103,1.0
1742159,9999,NaN,20201104,1.0
1742160,9999,NaN,20201105,1.0
1742161,9999,NaN,20201106,1.0


In [217]:
count_df = actual_df.groupby("service_id",as_index=False
                 ).sum(
                ).rename({"exception_type_x":"days_count"},axis=1)[["service_id","days_count"]]